In [14]:
!pip install gymnasium

In [15]:
import numpy as np
import gymnasium as gym
import os
import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from collections import deque
from IPython.display import Image
from matplotlib import animation

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### Initialize the environment

In [17]:
# Initialize the environment
env = gym.make('CartPole-v1') #render_mode="human"

state_space = env.observation_space.shape[0]
print('State Space:', state_space)
action_space = env.action_space.n
print('Action Space:', action_space)

State Space: 4
Action Space: 2


### Policy

In [18]:
# Policy Network
class Policy(nn.Module):
    def __init__(self , s_size , a_size , h_size ):
        super (Policy , self ).__init__ ()
        self.fc1 = nn.Linear( s_size , h_size )
        self.fc2 = nn.Linear( h_size , h_size * 2)
        self.fc3 = nn.Linear( h_size * 2, a_size )
    def forward(self , x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim =1)
    def act(self, state ):
        state = torch.from_numpy(state).float().unsqueeze(0)  #.to(device)
        probs = self.forward(state) # .cpu()
        m = Categorical(probs)
        # Random action
        action = m.sample()
        return action.item() , m.log_prob(action)

### REINFORCE
Initialize the policy parameter $\theta$ at random. <br>
**for** each episode $\{s_1, a_1, r_2, s_2, a_2, \dots, s_T\} $ **do** <br>
&nbsp; &nbsp; &nbsp; &nbsp;     **for** $t=1, 2, … , T $ **do** <br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Calculate the Return $G_t$ <br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Update policy parameters $\theta \leftarrow \theta + \alpha \gamma^t G_t \nabla_\theta \log \pi_\theta(a_t \vert s_t)$ <br>
&nbsp; &nbsp; &nbsp; &nbsp; **end for** <br>
**end for**

Visually Explained: https://towardsdatascience.com/reinforcement-learning-explained-visually-part-6-policy-gradients-step-by-step-f9f448e73754

In [19]:
# Training Function
def reinforce(
        policy ,
        optimizer ,
        n_training_episodes ,
        max_steps ,
        gamma ,
        print_every
        ):
    # scores_deque = deque(maxlen =100)
    scores = []

    # Each Episode
    for i_episode in range(1, n_training_episodes + 1):
        saved_log_probs = []
        rewards = []
        state = env.reset()[0]

        # t=1, 2, … , T (compute log(policy(a_t|s_t)))
        for t in range(max_steps):
            action , log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state , reward , done , _ , _ = env.step(action)
            rewards.append(reward)
            if done :
                break
        # scores_deque.append(sum( rewards ))
        scores.append(sum(rewards))

        returns = deque(maxlen = max_steps)
        n_steps = len(rewards)

        # List of discounted Returns (compute gamma^t*G_t)
        for t in range(n_steps)[:: -1]:
            disc_return_t = returns[0] if len(returns) > 0 else 0
            returns.appendleft(gamma*disc_return_t + rewards[t])

        # Total loss (disc_return = gamma^t*G_t; log_prob = log(policy(a_t|s_t)))
        policy_loss = []
        for log_prob , disc_return in zip( saved_log_probs , returns ):
            policy_loss.append(-log_prob * disc_return )
        policy_loss = torch.cat( policy_loss ).sum()

        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        print(" Episode {}, Reward : {}".format( i_episode ,sum(rewards)))

    return scores


In [20]:
# Hyperparameter
h_size = 128
lr = 0.001
n_training_episodes = 5000
max_steps = 500
gamma = 0.99

In [21]:
policy = Policy (
        s_size = state_space ,
        a_size = action_space ,
        h_size = h_size ,
        ) #.to(device)
optimizer = optim.Adam( policy.parameters() , lr=lr)

In [22]:
scores = reinforce (
        policy ,
        optimizer ,
        n_training_episodes ,
        max_steps ,
        gamma ,
        print_every = 100)

 Episode 1, Reward : 16.0
 Episode 2, Reward : 18.0
 Episode 3, Reward : 13.0
 Episode 4, Reward : 11.0
 Episode 5, Reward : 15.0
 Episode 6, Reward : 9.0
 Episode 7, Reward : 16.0
 Episode 8, Reward : 16.0
 Episode 9, Reward : 20.0
 Episode 10, Reward : 13.0
 Episode 11, Reward : 18.0
 Episode 12, Reward : 23.0
 Episode 13, Reward : 8.0
 Episode 14, Reward : 14.0
 Episode 15, Reward : 9.0
 Episode 16, Reward : 26.0
 Episode 17, Reward : 25.0
 Episode 18, Reward : 25.0
 Episode 19, Reward : 12.0
 Episode 20, Reward : 59.0
 Episode 21, Reward : 29.0
 Episode 22, Reward : 32.0
 Episode 23, Reward : 39.0
 Episode 24, Reward : 12.0
 Episode 25, Reward : 14.0
 Episode 26, Reward : 28.0
 Episode 27, Reward : 9.0
 Episode 28, Reward : 43.0
 Episode 29, Reward : 10.0
 Episode 30, Reward : 53.0
 Episode 31, Reward : 21.0
 Episode 32, Reward : 48.0
 Episode 33, Reward : 18.0
 Episode 34, Reward : 10.0
 Episode 35, Reward : 33.0
 Episode 36, Reward : 40.0
 Episode 37, Reward : 24.0
 Episode 38, R